In [1]:
from ipynb.fs.full.skill_to_l3_label_propagation_model import estimator_label_propagation

In [2]:
import sklearn

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np

In [4]:
result_folder = "C:\\data\\upwork1\\skill_to_l3_label_propagation_result\\"

# install vectic

## Install GCS packages
!pip install --q fsspec
!pip install --q gcsfs

#Install Vectice Python library 
# In this tutorial we will do code versioning using github, we also support gitlab
# and bitbucket: !pip install -q "vectice[github, gitlab, bitbucket]"
!pip install --q vectice[github]==22.3.5.1

#Verify if Vectice python library was installed
!pip3 show vectice

# configure vectice

In [7]:
#Import the required packages
from vectice import Experiment
from vectice.api.json import ModelType
from vectice.api.json import JobType
from vectice.api.json import JobArtifactType
from vectice.api.json import ModelVersionStatus
from vectice.api.json import VersionStrategy
import logging
import os
logging.basicConfig(level=logging.INFO)

# Specify the API endpoint for Vectice.
# You can specify your API endpoint here in the notebook, but we recommand you to add it to a .env file
os.environ['VECTICE_API_ENDPOINT']= "app.vectice.com"

# To use the Vectice Python library, you first need to authenticate your account using an API token.
# You can generate an API token from the Vectice UI, by going to the "API Tokens" section in the "My Profile" section
# which is located under your profile picture.
# You can specify your API Token here in the notebook, but we recommend you to add it to a .env file
os.environ['VECTICE_API_TOKEN'] = "6nKLaBqd9.3MYZXrl3GAbeNPQmj0Dy26nKLaBqd9O8aWRVvLogwpExBz4J7k"

# Add you project id. The project id can be found in the project settings page in the Vectice UI
project_id = 5863

# experiment 1: data

In [8]:
skill_to_l3_data_and_model_building = Experiment(
    job="skill_to_l3_data_and_model_building", 
    project=project_id, 
    job_type=JobType.PREPARATION, auto_code=True)

INFO:Client:Successfully authenticated. You'll be working on Project: Adaptive Skill Filtering part of Workspace: Sandbox - all


compatibility warning : Please upgrade to the Vectice SDK version 22.4.2 which is more compatible with your Vectice application.


INFO:Project:Job with id: 11172 successfully retrieved.


In [9]:
input_code = skill_to_l3_data_and_model_building.add_code_version_uri(
    git_uri="https://github.com/jingyanwang/skill_to_l3_example",
    entrypoint="skill_to_l3_label_propagation_vectice.ipynb") 

INFO:Project:CodeVersion with id: 2878 successfully created in Code vectice-examples.


In [10]:
skill_to_l3_data_and_model_building.add_dataset_version(
    dataset="skill_to_l3_data", 
    version_strategy=VersionStrategy.AUTOMATIC,
    )

ArtifactReference(code=None, dataset=skill_to_l3_data, model=None, version_number=None, version_id=None, version_name=None, version_strategy=VersionStrategy.AUTOMATIC, description=None, )

In [11]:
skill_to_l3_data_and_model_building.start(
    run_properties={
        "feature": "skill_to_skill_similarity", 
        "label": "skill_to_l3",
    },
    )

INFO:RunApi:Properties with names: ['feature', 'label'] successfully added to Run Run 2022-08-31T12:31:36.953229.
INFO:Job:Run with id: 19746 successfully created.


Run(id=19746, job=Job(name=skill_to_l3_data_and_model_building, id=11172, description=None, type=JobType.PREPARATION), name=Run 2022-08-31T12:31:36.953229, system_name=None, start_date=2022-08-31 08:31:42.609066, end_date=None, duration=None, status=RunStatus.STARTED, inputs=[ArtifactReference(code=1244, dataset=None, model=None, version_number=None, version_id=2878, version_name=None, version_strategy=VersionStrategy.MANUAL, description=None, ), ArtifactReference(code=None, dataset=skill_to_l3_data, model=None, version_number=None, version_id=None, version_name=None, version_strategy=VersionStrategy.AUTOMATIC, description=None, )], outputs=[])

# load the data and build the class instance

In [12]:
LP_model = estimator_label_propagation(
    skill_tag_id_src_npy_file = 'C:\\data\\upwork1\\src.npy',
    skill_tag_id_dst_npy_file = 'C:\\data\\upwork1\\dst.npy',
    transmission_probability_npy_file = 'C:\\data\\upwork1\\weight.npy',
    labeled_skill_l3_id_npy_file = 'C:\\data\\upwork1\\label.npy',
    labeled_skill_tag_id_npy_file = 'C:\\data\\upwork1\\node.npy',
    num_layers = 2,
    alpha = 0.01,
    use_gpu = False,
    )

# split training and test

In [13]:
training_idx, test_idx = train_test_split(
    LP_model.labeled_skill_id, 
    test_size = 0.1, 
    shuffle = True,
    )

In [14]:
# We create new dataset versions 
train_ds_version = skill_to_l3_data_and_model_building.add_dataset_version(
    dataset="skill_to_l3_data_train", 
    version_strategy=VersionStrategy.AUTOMATIC,
    )

test_ds_version = skill_to_l3_data_and_model_building.add_dataset_version(
    dataset="skill_to_l3_data_test", 
    version_strategy=VersionStrategy.AUTOMATIC)

# We complete the current experiment's run 
## The created dataset versions will be automatically attached as outputs of the run
skill_to_l3_data_and_model_building.complete()

# run the propagation algorithm

In [15]:
skill_to_l3_model_training = Experiment(
    job="skill_to_l3_model_training", 
    project=project_id, 
    job_type=JobType.TRAINING, 
    auto_code=True)

INFO:Client:Successfully authenticated. You'll be working on Project: Adaptive Skill Filtering part of Workspace: Sandbox - all


compatibility warning : Please upgrade to the Vectice SDK version 22.4.2 which is more compatible with your Vectice application.


INFO:Project:Job with id: 11170 successfully retrieved.


In [16]:
skill_to_l3_model_training.start(
    inputs=[input_code, train_ds_version, test_ds_version],
    run_properties={
        "alpha": 0.01, 
        "num_layers": 2,
    })

INFO:RunApi:Properties with names: ['alpha', 'num_layers'] successfully added to Run Run 2022-08-31T12:31:58.782674.
INFO:Job:Run with id: 19747 successfully created.


Run(id=19747, job=Job(name=skill_to_l3_model_training, id=11170, description=None, type=JobType.TRAINING), name=Run 2022-08-31T12:31:58.782674, system_name=None, start_date=2022-08-31 08:32:04.637647, end_date=None, duration=None, status=RunStatus.STARTED, inputs=[ArtifactReference(code=1244, dataset=None, model=None, version_number=None, version_id=2878, version_name=None, version_strategy=VersionStrategy.MANUAL, description=None, ), ArtifactReference(code=None, dataset=skill_to_l3_data_train, model=None, version_number=None, version_id=None, version_name=None, version_strategy=VersionStrategy.AUTOMATIC, description=None, ), ArtifactReference(code=None, dataset=skill_to_l3_data_test, model=None, version_number=None, version_id=None, version_name=None, version_strategy=VersionStrategy.AUTOMATIC, description=None, )], outputs=[])

In [17]:
print('start to run the algorithm with parameter: {}'.format(LP_model.get_params()))

running_time = LP_model.fit(
    training_idx.tolist(),
    )

print('running time: {}'.format(running_time))

start to run the algorithm with parameter: {'alpha': 0.01, 'num_layers': 2}
running time: 2.115912914276123


In [18]:
## test the model with the score method

auc_mirco = LP_model.score(
    test_idx.tolist(),
    )

print('auc over the test set: {}'.format(auc_mirco))

auc over the test set: 0.9140093883505644


In [19]:
metrics = {"auc_mirco": auc_mirco}
model_version = skill_to_l3_model_training.add_model_version(
    model="skill_to_l3", 
    algorithm="label_propagation", 
    metrics=metrics)

# We complete the current experiment's run 
## The created model version will be automatically attached as output of the run
skill_to_l3_model_training.complete()

INFO:Project:Model with id: 4816 successfully retrieved.
INFO:ModelVersionApi:Metrics with names: ['auc_mirco'] successfully added to ModelVersion Version 4.
INFO:Model:ModelVersion with id: 13182 successfully created in Model skill_to_l3.


# prepare the output 

In [20]:
skill_to_l3_model_inference = Experiment(
    job="skill_to_l3_model_inference", 
    project=project_id, 
    job_type=JobType.INFERENCE, 
    auto_code=True)

INFO:Client:Successfully authenticated. You'll be working on Project: Adaptive Skill Filtering part of Workspace: Sandbox - all


compatibility warning : Please upgrade to the Vectice SDK version 22.4.2 which is more compatible with your Vectice application.


INFO:Project:Job with id: 11171 successfully retrieved.


In [21]:
predictions = LP_model.predictions.to_sparse()

In [22]:
predictions.size()

torch.Size([14712, 241])

In [23]:
indices = predictions.indices()

In [24]:
skill_id = indices[0].cpu().numpy()
l3_id = indices[1].cpu().numpy()
score = predictions.values().cpu().numpy()

In [25]:
np.save(
  f'{result_folder}result_skill_id.npy',
  skill_id,  
  )

In [26]:
np.save(
  f'{result_folder}result_l3_id.npy',
  l3_id,  
  )

In [27]:
np.save(
  f'{result_folder}result_score.npy',
  score,  
  )

In [28]:
model_version = skill_to_l3_model_inference.add_model_version(
    model="skill_to_l3", 
    algorithm="label_propagation", 
    )

# We complete the current experiment's run 
## The created model version will be automatically attached as output of the run
skill_to_l3_model_inference.complete()

INFO:Project:Model with id: 4816 successfully retrieved.
INFO:Model:ModelVersion with id: 13183 successfully created in Model skill_to_l3.


# end